In [1]:
filename="input-ex.txt"
with open(filename) as f:
    lines = [line.rstrip().split(":") for line in f.readlines()]
    for i in range(len(lines)):
        line = lines[i]
        line[0] = line[0][10:]
        x_y = line[0].replace("x=","").replace("y=","").split(", ")
        line[0] = tuple(map(int,x_y))
        line[1] = tuple(map(int,line[1][22:].replace("x=","").replace("y=","").split(", ")))

sensors = [sensor[0] for sensor in lines]
beacons = [beacon[1] for beacon in lines]
combined = sensors+beacons

def print_map(sensors,beacons):
    combined = sensors+beacons
    min_x = min(combined, key=lambda tup: tup[0])[0]
    max_x = max(combined, key=lambda tup: tup[0])[0]
    min_y = min(combined, key=lambda tup: tup[1])[1]
    max_y = max(combined, key=lambda tup: tup[1])[1]

    for y in range(min_y,max_y):
        row_string = ""
        for x in range(min_x,max_x):
            if (x,y) in sensors:
                row_string += "S"
            elif (x,y) in beacons:
                row_string += "B"
            else:
                row_string += "."
        print(row_string)


In [41]:
#print("Map: ")
#print_map(sensors,beacons)
distances = []
for i in range(len(sensors)):
    distances.append(abs(sensors[i][0]-beacons[i][0])+abs(sensors[i][1]-beacons[i][1]))

min_x = min(combined, key=lambda tup: tup[0])[0]
max_x = max(combined, key=lambda tup: tup[0])[0]
#print(min_x,max_x)
row = 2000000
locations = 0 
row_string = ""

def manhatten(point1,point2):
    return abs(point1[0]-point2[0])+abs(point1[1]-point2[1])

all_positions = set([])
for i in range(len(sensors)):
    sensor = sensors[i]
    distance = distances[i]
    y_range = range(sensor[1]-distance,sensor[1]+distance)
    if row in y_range:
        intersection_distance = manhatten(sensor,(sensor[0],row))
        x_distance = distance-intersection_distance
        x_range = set(range(sensor[0]-x_distance,sensor[0]+x_distance+1))
        all_positions = all_positions.union(x_range)

for i in range(len(beacons)):
    x = beacons[i][0]
    if beacons[i][1] == row:
        if x in all_positions:
            all_positions.remove(x)

print(len(all_positions))
    
    

4961647


In [7]:
distances = []
for i in range(len(sensors)):
    distances.append(abs(sensors[i][0]-beacons[i][0])+abs(sensors[i][1]-beacons[i][1]))

locations = 0 
row_string = ""

def manhatten(point1,point2):
    return abs(point1[0]-point2[0])+abs(point1[1]-point2[1])

def get_border_points(point,distance):    
    points = []
    for x in range(point[0]-distance,point[0]+distance+1):
        for y in range(point[1]-distance,point[1]+distance+1):
            cur_dis = manhatten(point,(x,y))
            if cur_dis == distance:
                points.append((x,y))
    return points


search_range = 20
full_set = set(range(search_range+1))

locations = 0 
row_string = ""

def manhatten(point1,point2):
    return abs(point1[0]-point2[0])+abs(point1[1]-point2[1])

for row in range(20+1):
    all_positions = set([])
    for i in range(len(sensors)):
        sensor = sensors[i]
        distance = distances[i]
        y_range = range(sensor[1]-distance,sensor[1]+distance)
        if row in y_range:
            intersection_distance = manhatten(sensor,(sensor[0],row))
            x_distance = distance-intersection_distance
            x_range = set(range(sensor[0]-x_distance,sensor[0]+x_distance+1))
            all_positions = all_positions.union(x_range)
    reduced = full_set.intersection(all_positions)
    difference = full_set.difference(reduced)
    if len(difference) > 0:
        print(list(difference)[0]*4000000+row)
        break




Row  0

Row  1

Row  2

Row  3

Row  4

Row  5

Row  6

Row  7

Row  8

Row  9

Row  10

Row  11
56000011
28
